In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

video_path = "NeckPosture.avi" 
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file '{video_path}'")
    exit()

output_path = "processed_NeckPosture.avi" 
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get pose landmarks
    results = pose.process(rgb_frame)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
        right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y])
        nose = np.array([landmarks[mp_pose.PoseLandmark.NOSE].x,
                         landmarks[mp_pose.PoseLandmark.NOSE].y])

        # Calculate neck base
        neck_base = (left_shoulder + right_shoulder) / 2

        # Calculate angle
        neck_vector = nose - neck_base
        horizontal_vector = np.array([1, 0])
        angle = np.arctan2(neck_vector[1], neck_vector[0]) 
        angle_deg = np.abs(np.degrees(angle))

        cv2.putText(frame, f'Neck Angle: {angle_deg:.2f} deg', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    out.write(frame)

    cv2.imshow('Neck Posture Analysis', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved to '{output_path}'")

Error: Could not open video file 'NeckPosture.avi'
Processed video saved to 'processed_NeckPosture.avi'


I0000 00:00:1737683776.674525 7575562 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
OpenCV: Couldn't read video stream from file "NeckPosture.avi"
[ WARN:0@0.494] global cap.cpp:781 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:415: error: (-215:Assertion failed) !filename_pattern.empty() in function 'CvVideoWriter_Images'


